In [5]:
import pandas as pd
import os 
import import_ipynb
import mymodule as md

public = pd.read_csv('/projects/fmba_covid/hip_downsampled/pool/pool.aa.table.txt', sep = '\t')
public = public.drop(public[public.incidence < 20].index)
met_data = pd.read_csv('/projects/fmba_covid/hip_downsampled/metadata.txt', sep = '\t')

os.chdir("/projects/fmba_covid/hip_downsampled")

met_data = met_data.dropna(subset=['cmv', 'hla'])                         #удаляем NaN значения
sample_lst = [i for i in os.listdir() if i.startswith("HIP")]
sample_lst = md.IntersLst(list(met_data['file_name']), sample_lst)

print(sample_lst)


met_cmv = dict(zip(list(met_data['file_name']), list(met_data['cmv'])))  #Словарь "название образца" : "CMV"  
met_hla = dict(zip(list(met_data['file_name']), list(met_data['hla'])))  #Словарь "название образца" : "список из HLA"
cdr3_lst = list(public['cdr3aa'])                                            #Список ID CDR3aa
df_col = [1]*len(cdr3_lst)


#Это мой датафрейм
df = pd.DataFrame({'CMV+_HLA-B*35' : df_col, 'CMV+_other' : df_col, 'CMV-_HLA-B*35' : df_col, 'CMV-_other' : df_col}, 
                  index = cdr3_lst)
df.index.name='cdr3aa'
#Это мой датафрейм


hla = "HLA-B*35" 

for filename in sample_lst:
    #if filename.startswith("HIP"):
        sample = md.read_col(filename, 3)
        found = md.IntersLst(cdr3_lst, sample)        
        if met_cmv[filename]== '+':
            if hla in list(met_hla[filename].split(',')):
                df.loc[found, 'CMV+_HLA-B*35'] += 1
            else:
                df.loc[found, 'CMV+_other'] += 1
        elif met_cmv[filename]== '-':
            if hla in list(met_hla[filename].split(',')):
                df.loc[found, 'CMV-_HLA-B*35'] += 1
            else: 
                df.loc[found, 'CMV-_other'] += 1
    
os.chdir("/home/daria/A02/common")    
#df.to_csv('cmv_hla-B*35_2.txt', sep = '\t')


KeyboardInterrupt: 

In [37]:
import pandas as pd
import import_ipynb
from mymodule import OddsRario_Pvalue
import os 
import scipy.stats as stats
import statsmodels.stats.multitest as smt

cmv_hla = pd.read_csv('/home/daria/TCR-association/common/cmv_hla-B*35_2.txt', sep='\t')
cmv_hla.columns = ['cdr3aa','CMV+_HLA-B*35', 'CMV+_other', 'CMV-_HLA-B*35', 'CMV-_other']
cmv_hla = cmv_hla.set_index('cdr3aa')

md.OddsRario_Pvalue(cmv_hla, 'CMV+_HLA-B*35', 'CMV+_other', 'CMV-_HLA-B*35', 'CMV-_other')

cmv_hela = cmv_hela.drop(cmv_hela[cmv_hela.odds_ratio < 1.1].index) 
df.to_csv('cmv_hla_pv_1.txt', sep = '\t')
df_finish = cmv_hela.sort_values('p-value', kind='mergesort')

#df_finish.to_csv('cmv_hla-B*35_pv_2.txt', sep = '\t')
print(df_finish)

NameError: name 'stats' is not defined

In [4]:
import pandas as pd
import import_ipynb
import mymodule as md
import os 
import scipy.stats as stats
#import statsmodels.stats.multitest as smt

cmv_hla = pd.read_csv('/home/daria/TCR-association/common/cmv_hla-B*35_2.txt', sep='\t')
cmv_hla.columns = ['cdr3aa','CMV+_HLA-B*35', 'CMV+_other', 'CMV-_HLA-B*35', 'CMV-_other']
cmv_hla = cmv_hla.set_index('cdr3aa')


def OddsRario_Pvalue(df, col1, col2, col3, col4):
    index_lst = list(df.index) 
    df['odds_ratio'] = [0]*len(index_lst)
    df['p-value'] = [0]*len(index_lst)
    for i in index_lst:
        a = df.loc[i, col1]
        b = df.loc[i, col2]
        c = df.loc[i, col3]
        d = df.loc[i, col4]
        сont_lst = [[a,b], [c,d]]
        oddsratio, pvalue = stats.fisher_exact(сont_lst)
        df.loc[i, 'odds_ratio'] = oddsratio
        df.loc[i, 'p-value'] =  pvalue



OddsRario_Pvalue(cmv_hla, 'CMV+_HLA-B*35', 'CMV+_other', 'CMV-_HLA-B*35', 'CMV-_other')

cmv_hla = cmv_hla.drop(cmv_hla[cmv_hla.odds_ratio < 1.1].index) 
df_finish = cmv_hla.sort_values('p-value', kind='mergesort')

os.chdir("/home/daria/TCR-association/common")
df_finish.to_csv('cmv_hla-B*35_pv_2.txt', sep = '\t')


str